In [3]:
# import packages needed
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import tkinter
import json
from textblob import TextBlob
import sys   
import re  

In [4]:
# scrap stock
def stock_info(name):
    stock_data = {}
    url_smy = 'https://www.nasdaq.com/symbol/' + str(name)
    web_data_smy = requests.get(url_smy)
    soup_smy = BeautifulSoup(web_data_smy.text, 'lxml')
    urls_smy_price = soup_smy.select('div #qwidget_lastsale')
    stock_price = str(urls_smy_price).replace('>','<').split('<')[2]

    urls_smy_trend = soup_smy.select('div #qwidget-arrow')
    trend_color = str(urls_smy_trend).replace('"','-').split('-')[5]
    
    stock_data = {'company': name, 'price' : stock_price, 'trend' : trend_color}
    return stock_data

In [5]:
def stock_history(name):
    url_stkhis = 'https://www.nasdaq.com/symbol/' + str(name) + '/historical'
    web_data_stkhis = requests.get(url_stkhis)
    soup_stkhis = BeautifulSoup(web_data_stkhis.text, 'lxml')
    stkhis_price = str(soup_stkhis.select('div #quotes_content_left_pnlAJAX td')).split()
    temp = []
    for i in range(len(stkhis_price)):
        if (i % 3 == 1):
            temp.append(stkhis_price[i])
    date = []
    open_price = []
    for j in range(len(temp)):
        if (j % 6 == 0):
            date.append(temp[j])
        if (j % 6 == 1):
            open_price.append(temp[j])
    return (date,open_price)

In [6]:
# oxy news
def news_info(name):
    news_data = {}
    url_news = 'https://www.nasdaq.com/symbol/' + str(name) + '/news-headlines'
    web_data_news = requests.get(url_news)
    soup_news = BeautifulSoup(web_data_news.text, 'lxml')
    urls_all_news = soup_news.select('span.fontS14px > a') 
    # get all information about news including titles and urls

    url_news_host = 'https://www.nasdaq.com/article/'
    news_url = [] # store all news url in here
    for link in urls_all_news:
        if link.text.isspace():
            continue
        else:
            page_url = url_news_host + link.get('href')
            news_url.append(page_url)

    news_list = str(urls_all_news).replace('">','</a>').split('</a>')
    num = len(news_list)
    news_title = [] # store all news title in here
    for n in range(num):
        if (n % 2 == 1):
            news_title.append(news_list[n])
    
    polarity = []
    for link in news_url:
        web_data_url = requests.get(link)
        soup_url = BeautifulSoup(web_data_url.text, 'lxml')
        # soup_url
        article = str(soup_url.select('div #articleText'))
        article = re.sub('<.*?>', "", article)
        blob = TextBlob(article)
        polarity.append(blob.sentiment.polarity)

    return news_url,news_title, polarity
    
#     news_data = {'name': list(np.repeat(name,num)), 'titles': news_title, 'url': news_url}

In [7]:
# oxy marketcap
def marketcap_info(name):
    url_smy = 'https://www.nasdaq.com/symbol/' + name
    web_data_smy = requests.get(url_smy)
    soup_smy = BeautifulSoup(web_data_smy.text, 'lxml')
    urls_smy_price = soup_smy.select('div.table-cell')
    str_temp = str(urls_smy_price).replace('</b>','<b>').split('<b>')
    loc_mc = str_temp.index('Market Cap')
    market_cap = str_temp[loc_mc + 1].split('\r\n')[1].replace(" ","")
    return market_cap

# oxy cash flow
def cashflow_info(name):
    url_smy = 'https://www.nasdaq.com/symbol/' + name + '/financials?query=income-statement'
    web_data_smy = requests.get(url_smy)
    soup_smy = BeautifulSoup(web_data_smy.text, 'lxml')
    if len(soup_smy.select('tr.net > td')) > 0:
        cash_flow = list(soup_smy.select('tr.net > td')[1])
    else:
        cash_flow = ['None']
    return cash_flow

In [9]:
def sentiment(name):
    news_url, news_list = news_info(name)
    polarity = []
    for link in news_url:
        web_data_url = requests.get(link)
        soup_url = BeautifulSoup(web_data_url.text, 'lxml')
        # soup_url
        article = str(soup_url.select('div #articleText'))
        article = re.sub('<.*?>', "", article)
        blob = TextBlob(article)
        polarity.append(blob.sentiment.polarity)
    return polarity

In [10]:
def extract_stock_history(this_stock_history):
    dates = this_stock_history[0]
    prices = this_stock_history[1]
    dates.pop(0)
    prices.pop(0)
    res = {"date":dates, "price":prices}
    return res

In [11]:
def extract_news_info(this_news_info):
    titles = this_news_info[1]
    urls = this_news_info[0]
    polarity = list(map(lambda x : None if x == 0 else x, this_news_info[2]))
    res = {"title":titles, "url":urls, "polarity":polarity}
    
    for i in range(len(titles)):
        res["url"][i] = res["url"][i][31:]
    return res


In [12]:
def toDict(comp):
#      done
    this_stock_history = extract_stock_history(stock_history(comp))
#      to be done
    this_stock_info = stock_info(comp)
#      done
    this_news_info = extract_news_info(news_info(comp))
    this_market_cap = int(marketcap_info(comp).replace(",", ""))
    if cashflow_info(comp)[0] == "None":
        this_cashflow = None
    else:
        this_cashflow = int(cashflow_info(comp)[0].replace(",", "").replace("$", ""))
    
    res = {}
    res["company"] = comp
    res["stock_price"] = this_stock_history
    res["article"] = this_news_info
    res["market_cap"] = this_market_cap
    res["net_cash_flow"] = this_cashflow
    return res
    


In [13]:
toDict('oxy')

{'company': 'oxy',
 'stock_price': {'date': ['02/25/2019',
   '02/22/2019',
   '02/21/2019',
   '02/20/2019',
   '02/19/2019',
   '02/15/2019',
   '02/14/2019',
   '02/13/2019',
   '02/12/2019',
   '02/11/2019',
   '02/08/2019',
   '02/07/2019',
   '02/06/2019',
   '02/05/2019',
   '02/04/2019',
   '02/01/2019',
   '01/31/2019',
   '01/30/2019',
   '01/29/2019',
   '01/28/2019',
   '01/25/2019',
   '01/24/2019',
   '01/23/2019',
   '01/22/2019',
   '01/18/2019',
   '01/17/2019',
   '01/16/2019',
   '01/15/2019',
   '01/14/2019',
   '01/11/2019',
   '01/10/2019',
   '01/09/2019',
   '01/08/2019',
   '01/07/2019',
   '01/04/2019',
   '01/03/2019',
   '01/02/2019',
   '12/31/2018',
   '12/28/2018',
   '12/27/2018',
   '12/26/2018',
   '12/24/2018',
   '12/21/2018',
   '12/20/2018',
   '12/19/2018',
   '12/18/2018',
   '12/17/2018',
   '12/14/2018',
   '12/13/2018',
   '12/12/2018',
   '12/11/2018',
   '12/10/2018',
   '12/07/2018',
   '12/06/2018',
   '12/04/2018',
   '12/03/2018',
   '11

In [148]:
# this_stock_history = stock_history("OXY")
# extract_stock_history(this_stock_history)
# this_news_info = news_info("oxy")
# extract_news_info(this_news_info)

# marketcap_info("OXY")
# cashflow_info("OXY")

import json

comp_list = ["OXY", "EOG", "APC", "APA", "COP", "PXD"]
res = {}
for comp in comp_list:
    res[comp] = toDict(comp)

output = json.dumps(res)
f = open("data.json", "w")
f.write(output)
f.close()